In [3]:
import numpy as np
import pandas as pd

import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [4]:
# standard imports
import pandas as pd
from sklearn.model_selection import RepeatedKFold

# some helpful imports from sklearndf
from sklearndf.pipeline import RegressorPipelineDF
from sklearndf.regression import RandomForestRegressorDF

# relevant FACET imports
from facet.data import Sample
from facet.selection import LearnerRanker, LearnerGrid

In [5]:
SINAN_PATH      = './../data/raw_data/SINAN_prep_05.csv'
MUNICIPIOS_PATH = './../data/municipios_prep_05.csv'
BOLSA_PATH = './../data/consolidada_bolsafamilia.csv'
INEP_PATH = './../data/consolidada_inep.csv'
ATLAS_PATH = './../data/atlas_desenvolvimento_humano_por_municipio.csv'
OCUPACOES_PATH  = './../data/cbo_ocupacoes.csv'
PNAD_PATH  = './../data/PNAD_consolidado.csv'
MAPBOX_TOKEN    = 'pk.eyJ1IjoibHVjYXNuc2VxIiwiYSI6ImNrb241dHZ0cTBpd2MycW5yMGp2enFtMmkifQ.N6NJGlWhG-iYrIJMQ1MVVw'

px.set_mapbox_access_token(MAPBOX_TOKEN)

In [6]:
municipios_df = pd.read_csv(MUNICIPIOS_PATH)

In [7]:
municipios_df.head()

,sinan_id,id,nome,uf_nome,uf_id,uf,regiao,latitude,longitude,pop_2013,...,munic_y19_mtic012,munic_y19_mtic013,munic_y19_mtic014,munic_y19_mtic015,munic_y19_mtic016,munic_y19_mtic017,munic_y19_mgov04,munic_y19_mseg01,munic_y19_mseg17.1,munic_y19_mdhu556
0,110001,1100015,Alta Floresta D'Oeste,Rondônia,11,RO,Norte,-11.929692,-61.996128,25728,...,Não,Não,Sim,Sim,Sim,Não,Sim,Não possui estrutura,Não,Não
1,110002,1100023,Ariquemes,Rondônia,11,RO,Norte,-9.907652,-63.033069,101269,...,Não,Sim,Sim,Sim,Sim,Sim,Sim,Secretaria municipal em conjunto com outras po...,Sim,Não
2,110003,1100031,Cabixi,Rondônia,11,RO,Norte,-13.505215,-60.677357,6495,...,Não,Não,Não,Sim,Não,Não,Não,Não possui estrutura,Não,Não
3,110004,1100049,Cacoal,Rondônia,11,RO,Norte,-11.433300,-61.442050,85863,...,Não,Não,Sim,Sim,Não,Não,Sim,Não possui estrutura,Sim,Não
4,110005,1100056,Cerejeiras,Rondônia,11,RO,Norte,-13.183969,-61.204778,18041,...,Não,Não,Sim,Sim,Sim,Não,Sim,Não possui estrutura,Não,Não


In [8]:
for c in municipios_df.columns:
    print(c)

sinan_id
id
nome
uf_nome
uf_id
uf
regiao
latitude
longitude
pop_2013
pop_2014
pop_2015
pop_2016
pop_2017
unit_count_2013
unit_relat_2013
denun_count_2013
denun_relat_2013
unit_count_2014
unit_relat_2014
denun_count_2014
denun_relat_2014
unit_count_2015
unit_relat_2015
denun_count_2015
denun_relat_2015
unit_count_2016
unit_relat_2016
denun_count_2016
denun_relat_2016
unit_count_2017
unit_relat_2017
denun_count_2017
denun_relat_2017
cluster
denun_sexismo_count_2013
denun_sexismo_relat_2013
denun_maioridade_count_2013
denun_maioridade_relat_2013
denun_relacao_count_2013
denun_relacao_relat_2013
denun_sexismo_count_2014
denun_sexismo_relat_2014
denun_maioridade_count_2014
denun_maioridade_relat_2014
denun_relacao_count_2014
denun_relacao_relat_2014
denun_sexismo_count_2015
denun_sexismo_relat_2015
denun_maioridade_count_2015
denun_maioridade_relat_2015
denun_relacao_count_2015
denun_relacao_relat_2015
denun_sexismo_count_2016
denun_sexismo_relat_2016
denun_maioridade_count_2016
denun_maior

In [9]:
municipios_df.shape

(5570, 545)

In [ ]:
def get_encoder(values):
    
    encoder = LabelEncoder()
    encoded = encoder.fit_transform(values.reshape(-1, 1))
    
    return encoded, encoder

In [ ]:
fig = px.histogram(municipios_df, x='pop_2017', marginal="box")
fig.show()

In [ ]:
df = municipios_df[(municipios_df['pop_2017'] >= 10000) &  (municipios_df['pop_2017'] <= 500000)]
fig = px.histogram(df, x='denun_relat_2017', marginal="box")
fig.show()

### Pipeline de preparação dos dados de treino

In [ ]:
def remove_munic_nan_indexes(dataframe):
    
    munic_cols = []

    for col in dataframe.columns:
        if 'munic' in col:
            munic_cols.append(col)
            
    for i, col in enumerate(munic_cols):
        
        if i == 0:
            df = dataframe[dataframe[col].notna()]
        else:
            df = df[df[col].notna()]
    
    return df

In [ ]:
def train_test_data_pipeline(dataframe, years, train_size = 0.7, filter_pop = True):
    '''
    Retorna X_df_train, Y_df_train, X_df_test e Y_df_test
    '''
    
    values_df = dataframe.set_index('id')
    
    # Removendo valores NaN do MUNIC
    values_df = remove_munic_nan_indexes(values_df)
    
    # Colunas do munic
    munic_cols = []

    for col in municipios_df.columns:
        if 'munic' in col:
            munic_cols.append(col)
            
    for col in ['munic_y13_a4', 'munic_y13_a21', 'munic_y13_a233', 'munic_y13_a240']:
        munic_cols.remove(col)
    
    # Features
    feature_columns = ['regiao', 'uf'] + munic_cols
    feature_columns = ['regiao', 'uf']
    
    encodeds, encoders = {}, {}
    for column in feature_columns:

        encodeds[column], encoders[column] = get_encoder(values_df[column].values)
        values_df[column] = encodeds[column]
        
    # Estáticas
    estatic_columns = ['latitude', 'pnad_espvida', 'pnad_rdpc','pnad_t_analf25m','pnad_t_freq15a17',
                       'pnad_t_freq5a6','pnad_t_freq6a14','pnad_anosest','pnad_mort1','pnad_razdep','pnad_gini',
                       'pnad_prentrab','pnad_pind','pnad_rmpob'] + feature_columns
    
    # Variáveis
    variable_prefix = ['pop', 'renda', 'share', 'percentual', 'denun_relat']
    
    variable_prefix_skip = ['faixa_60_120', 'faixa_0_13', 'faixa_14_17', 'cbo_grupo_profissiona', 'ef i comp', 'ef ii incom',
                            'membros superior', 'membros das forças', 'trabalho infantil', 'sit_conjug_solteir', 'ssor_cuidador',
                            'ssor_irmão', 'motiv_defici', 'media_median', 'sexo_i', 'violence_legal', 'trabalhadores em serviços']
    
    year_columns = []
    for col in values_df.columns:
        skip = False
        for skip_prefix in variable_prefix_skip:
            if skip_prefix in col:
                skip = True
                break
        if not skip:
            for prefix in variable_prefix:
                if prefix in col:
                    year_columns.append(col)
                    
    # Removendo sufixo de ano "_year"
    renamed_year_columns_map = {}
    for col in year_columns:
        renamed_year_columns_map[col] = '_'.join(col.split('_')[:-1])
        
    keep_columns = year_columns + estatic_columns
    
    df = pd.DataFrame()
    
    min_population = 10000
    max_population = 500000
    
    for year in years:
        
        # Selecionando intervalo de população
        
        if filter_pop:
            temp_df = values_df[(values_df[f'pop_{year}'] >= min_population) & (values_df[f'pop_{year}'] <= max_population)]
        else:
            temp_df = values_df.copy()
        
        year_df = pd.DataFrame(index = temp_df.index)
        for col in estatic_columns:
            year_df[col] = temp_df[col]
            
        for col in year_columns:
            if str(year) in col:
                year_df[col] = temp_df[col]
                
        year_df = year_df.rename(renamed_year_columns_map, axis=1)
        
        year_df = year_df.rename(lambda x : f'{x}_{year}', axis=0)
        
        year_df = year_df.fillna(-1)
        
        df = pd.concat((df, year_df))
    
    X_df = df.drop(f'denun_relat', axis=1)
    Y_df = df['denun_relat']
    
    ####### tirar uma amostra das colunas
    
    # Separando df em train/test
    train_indexes = list(np.random.choice(X_df.index, int(len(X_df)*train_size), replace = False))
    test_indexes  = []
    
    for index in X_df.index:
        if index not in train_indexes:
            test_indexes.append(index)
    
    X_df_train = X_df.drop(test_indexes, axis=0)
    Y_df_train = Y_df.drop(test_indexes, axis=0)
    X_df_test  = X_df.drop(train_indexes, axis=0)
    Y_df_test  = Y_df.drop(train_indexes, axis=0)
    
    return X_df_train, Y_df_train, X_df_test, Y_df_test

In [ ]:
def visualize_train_result(X_df, Y_df, regressor, original_df, years):
    
    X_values = X_df.values
    Y_values = Y_df.values.reshape(-1, 1)
    
    predictions_df = pd.DataFrame()
    
    predictions_df['prediction'] = regressor.predict(X_values).reshape(-1,)
    predictions_df['real']       = Y_values.reshape(-1,)

    
    predictions_df['abs_diff']   = np.abs(predictions_df['prediction'] - predictions_df['real'])
    predictions_df['diff']       = predictions_df['prediction'] - predictions_df['real']
    predictions_df['sqr_diff']   = (predictions_df['prediction'] - predictions_df['real'])**2
    
    variables           = ['nome', 'uf', 'regiao', 'pop', 'year']
    variables_values    = {}
    
    original_df_indexed = original_df.set_index('id')
    
    for variable in variables:
        variables_values[variable] = []
        
    for index in X_df.index:
        
        index_parts = index.split('_')
        mun_id, year = int(index_parts[0]), int(index_parts[1])

        mun_data = original_df_indexed.loc[mun_id]
        
        for variable in variables:
            
            if variable == 'pop':
                variables_values[variable].append(mun_data[f'pop_{year}'])
            elif variable == 'year':
                variables_values[variable].append(year)
            else:
                variables_values[variable].append(mun_data[variable])
    
    for variable in variables:
        predictions_df[variable] = variables_values[variable]
        
    score = regressor.score(X_values, Y_values)
    name  = str(regressor).split('()')[0]
    
    title = f'Regressor: {name} | score: {round(score, 5)}'
    
    scaler = MinMaxScaler()
    
    size   = scaler.fit_transform(predictions_df['year'].values.reshape(-1, 1)).reshape(-1)
    col_year = predictions_df['year'].apply(lambda x : f'y_{x}')

    fig = px.scatter(predictions_df, x='prediction', y='real', color='diff', size=size, size_max=7,
                     hover_data=variables + ['abs_diff', 'diff', 'sqr_diff'], title=title)
    
    max_val = np.max(predictions_df['prediction'].values)
    min_val = np.min(predictions_df['prediction'].values)
    
    fig.add_trace(go.Line(x=np.linspace(min_val*0.9,max_val*1.1), y=np.linspace(min_val*0.9,max_val*1.1)))

    return fig

### Aplicando alguns regressores

#### Linear Regression

In [ ]:
%%time

years = [2013, 2014, 2015, 2016, 2017]

X_df_train, Y_df_train, X_df_test, Y_df_test = train_test_data_pipeline(municipios_df, years, train_size = 0.7, filter_pop = False)

X_values_train, Y_value_train = X_df_train.values, Y_df_train.values.reshape(-1,)
X_values_test, Y_values_test  = X_df_test.values, Y_df_test.values.reshape(-1,)

lr_regressor = LinearRegression()

lr_regressor = lr_regressor.fit(X_values_train, Y_value_train)
score     = lr_regressor.score(X_df_test, Y_df_test)

print(f'Train Size: {round(100*len(X_df_train)/(len(municipios_df)*len(years)), 2)}% | {len(X_df_train)}/{len(municipios_df)*len(years)}')
print('Score:', round(score, 4))

In [ ]:
fig = visualize_train_result(X_df_test, Y_df_test, lr_regressor, municipios_df, years)
fig.show()

### Polynomial Features (LR)

In [ ]:
# %%time

# years = [2013, 2014, 2015, 2016, 2017]

# X_df_train, Y_df_train, X_df_test, Y_df_test = train_test_data_pipeline(municipios_df, years, train_size = 0.7)

# X_values_train, Y_value_train = X_df_train.values, Y_df_train.values.reshape(-1,)
# X_values_test, Y_values_test  = X_df_test.values, Y_df_test.values.reshape(-1,)

# dim = 3
# poly = PolynomialFeatures(dim)

# X_values_train = poly.fit_transform(X_values_train)
# X_values_test  = poly.transform(X_values_test)

# lr_poly_regressor = LinearRegression()

# lr_poly_regressor = lr_poly_regressor.fit(X_values_train, Y_value_train)
# score             = lr_poly_regressor.score(X_values_test, Y_df_test)

# print(f'Train Size: {round(100*len(X_df_train)/(len(municipios_df)*len(years)), 2)}% | {len(X_df_train)}/{len(municipios_df)*len(years)}')
# print('Score:', round(score, 4))

### Random Forest Regressor

In [ ]:
%%time

years = [2013, 2014, 2015, 2016, 2017]

X_df_train, Y_df_train, X_df_test, Y_df_test = train_test_data_pipeline(municipios_df, years, train_size = 0.7, filter_pop = False)

X_values_train, Y_value_train = X_df_train.values, Y_df_train.values.reshape(-1,)
X_values_test, Y_values_test  = X_df_test.values, Y_df_test.values.reshape(-1,)

rf_regressor = RandomForestRegressor()

rf_regressor = rf_regressor.fit(X_values_train, Y_value_train)
score     = rf_regressor.score(X_df_test, Y_df_test)

print(f'Train Size: {round(100*len(X_df_train)/(len(municipios_df)*len(years)), 2)}% | {len(X_df_train)}/{len(municipios_df)*len(years)}')
print('Score:', round(score, 4))

In [ ]:
fig = visualize_train_result(X_df_test, Y_df_test, rf_regressor, municipios_df, years)
fig.show()

### Gradient Boosting Regressor

In [ ]:
%%time

years = [2013, 2014, 2015, 2016, 2017]

X_df_train, Y_df_train, X_df_test, Y_df_test = train_test_data_pipeline(municipios_df, years, train_size = 0.7, filter_pop = False)

X_values_train, Y_value_train = X_df_train.values, Y_df_train.values.reshape(-1,)
X_values_test, Y_values_test  = X_df_test.values, Y_df_test.values.reshape(-1,)

xg_regressor = GradientBoostingRegressor()

xg_regressor = xg_regressor.fit(X_values_train, Y_value_train)
score        = xg_regressor.score(X_df_test, Y_df_test)

print(f'Train Size: {round(100*len(X_df_train)/(len(municipios_df)*len(years)), 2)}% | {len(X_df_train)}/{len(municipios_df)*len(years)}')
print('Score:', round(score, 4))

In [ ]:
fig = visualize_train_result(X_df_test, Y_df_test, xg_regressor, municipios_df, years)
fig.show()

#### Avaliando Capitais

In [ ]:
years = [2013, 2014]

capitals = ['Rio Branco','Maceió','Macapá','Manaus','Salvador','Fortaleza','Brasília','Vitória',
            'Goiânia','São Luís','Cuiabá','Campo Grande','Belo Horizonte','Belém','João Pessoa',
            'Curitiba','Recife','Teresina','Rio de Janeiro','Natal','Porto Alegre','Porto Velho',
            'Boa Vista','Florianópolis','São Paulo','Aracaju','Palmas']

mun_data = municipios_df[municipios_df['nome'].isin(capitals)]

X_df, Y_df, _, _ = train_test_data_pipeline(mun_data, years, train_size = 1, filter_pop = False)

In [ ]:
fig = visualize_train_result(X_df, Y_df, lr_regressor, municipios_df, years)
fig.show()

In [ ]:
fig = visualize_train_result(X_df, Y_df, rf_regressor, municipios_df, years)
fig.show()

In [ ]:
fig = visualize_train_result(X_df, Y_df, xg_regressor, municipios_df, years)
fig.show()

#### Avaliando base toda

In [ ]:
X_df, Y_df, _, _ = train_test_data_pipeline(municipios_df, years, train_size = 1, filter_pop = False)

In [ ]:
fig = visualize_train_result(X_df, Y_df, lr_regressor, municipios_df, years)
fig.show()

In [ ]:
fig = visualize_train_result(X_df, Y_df, rf_regressor, municipios_df, years)
fig.show()

In [ ]:
fig = visualize_train_result(X_df, Y_df, xg_regressor, municipios_df, years)
fig.show()

#### Colunas usadas no treinamento

In [ ]:
len(X_df.columns)

### Facet Exploration

In [ ]:
years = [2013,2015,2017]

X_df_train, Y_df_train, _, _ = train_test_data_pipeline(municipios_df, years, train_size = 1)

df = X_df_train.copy()
df['denun_relat'] = Y_df_train.values

df.head(3)

In [ ]:
# create FACET sample object
df_sample = Sample(observations=df, target_name="denun_relat")

# create a (trivial) pipeline for a random forest regressor
rnd_forest_reg = RegressorPipelineDF(
    regressor=RandomForestRegressorDF(n_estimators=200, random_state=42)
)

# define grid of models which are "competing" against each other
rnd_forest_grid = [
    LearnerGrid(
        pipeline=rnd_forest_reg,
        learner_parameters={
            "min_samples_leaf": [8, 11],
            "max_depth": [10, 12],
        }
    ),
]

# create repeated k-fold CV iterator
rkf_cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

# rank your candidate models by performance (default is mean CV score - 2*SD)
ranker = LearnerRanker(
    grids=rnd_forest_grid, cv=rkf_cv, n_jobs=-3
).fit(sample=df_sample)

# get summary report
ranker.summary_report()

In [ ]:
# fit the model inspector
from facet.inspection import LearnerInspector
inspector = LearnerInspector(n_jobs=-3)
inspector.fit(crossfit=ranker.best_model_crossfit_)

In [ ]:
# visualise synergy as a matrix
from pytools.viz.matrix import MatrixDrawer
synergy_matrix = inspector.feature_synergy_matrix()
px.imshow(synergy_matrix, width = 2500, height = 2500, title = 'Sinergy Matrix')

In [ ]:
# visualise redundancy as a matrix
redundancy_matrix = inspector.feature_redundancy_matrix()
px.imshow(redundancy_matrix, width = 2500, height = 2500, title = 'Redudancy Matrix')

In [ ]:
# visualise redundancy using a dendrogram
from pytools.viz.dendrogram import DendrogramDrawer
import matplotlib.pyplot as plt
redundancy = inspector.feature_redundancy_linkage()

plt.figure(figsize=(14,40))
DendrogramDrawer().draw(data=redundancy, title="Redundancy Dendrogram")

In [ ]:
keep_columns = ['denun_relat_2016', 'denun_maioridade_relat_2016', 'denun_relacao_2016', 
                'denun_relat_2015', 'denun_maioridade_relat_2015', 'denun_relacao_2015',
                'denun_count_2015', 'denun_relat_2014', 'denun_relat_2013', 'denun_count_2014',
                'renda_media_sum_2013', 'renda_media_sum_2016', 'latitude', 'regiao', 
                'share_agua_canalizada_2013', 'share_agua_canalizada_2016', 'renda_media_median_2013', 
                'renda_media_median_2016']

In [ ]:
# FACET imports
from facet.validation import BootstrapCV
from facet.crossfit import LearnerCrossfit
from facet.simulation import UnivariateUpliftSimulator
from facet.data.partition import ContinuousRangePartitioner
from facet.simulation.viz import SimulationDrawer

# create bootstrap CV iterator
bscv = BootstrapCV(n_splits=1000, random_state=42)

# create a bootstrap CV crossfit for simulation using best model
boot_crossfit = LearnerCrossfit(
    pipeline=ranker.best_model_,
    cv=bscv,
    n_jobs=-3,
    verbose=False,
).fit(sample=df_sample)

SIM_FEAT = "denun_relat_2016"
simulator = UnivariateUpliftSimulator(crossfit=boot_crossfit, n_jobs=-3)

# split the simulation range into equal sized partitions
partitioner = ContinuousRangePartitioner()

# run the simulation
simulation = simulator.simulate_feature(feature_name=SIM_FEAT, partitioner=partitioner)

# visualise results
plt.figure(figsize=(14,12))
SimulationDrawer().draw(data=simulation, title=SIM_FEAT)